In [0]:
%python
import boto3
import pandas as pd
from io import StringIO
import time

In [0]:
%python
athena_client=boto3.client(
    "athena",
    aws_access_key_id= AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION
)

In [0]:
Dict={}
def download_and_load_query_results(
    client: boto3.client,query_response: Dict
)-> pd.DataFrame:
    while True:
        try:
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as e:
            if "not yet finished" in str(e):
                time.sleep(0.001)
            else:
                raise e
    temp_file_location: str="anthena_query_results.csv"
    
    s3_client=boto3.client(
        "s3",
        aws_access_key_id= AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
    )
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    
    )
    return pd.read_csv(temp_file_location)

In [0]:
response=athena_client.start_query_execution(
    QueryString="SELECT * FROM engima_jhu",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    }
)
engima_jhu=download_and_load_query_results(athena_client,response)

response=athena_client.start_query_execution(
    QueryString="SELECT * FROM county_population",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    }
)
county_population=download_and_load_query_results(athena_client,response)

response=athena_client.start_query_execution(
    QueryString="SELECT * FROM country_codes",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    }
)
country_codes=download_and_load_query_results(athena_client,response)

response=athena_client.start_query_execution(
    QueryString="SELECT * FROM us",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    }
)
us=download_and_load_query_results(athena_client,response)

response=athena_client.start_query_execution(
    QueryString="SELECT * FROM us_county",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    }
)
us_county=download_and_load_query_results(athena_client,response)

response=athena_client.start_query_execution(
    QueryString="SELECT * FROM us_daily",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    }
)
us_daily=download_and_load_query_results(athena_client,response)

response=athena_client.start_query_execution(
    QueryString="SELECT * FROM us_states",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    }
)
us_states=download_and_load_query_results(athena_client,response)

response=athena_client.start_query_execution(
    QueryString="SELECT * FROM us_states_abbr",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    }
)
us_states_abbr=download_and_load_query_results(athena_client,response)

response=athena_client.start_query_execution(
    QueryString="SELECT * FROM us_states_daily",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    }
)
us_states_daily=download_and_load_query_results(athena_client,response)

response=athena_client.start_query_execution(
    QueryString="SELECT * FROM usa_hospital_beds",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    }
)
us_hospital_beds=download_and_load_query_results(athena_client,response)


In [0]:
header=us_states_abbr.iloc[0]
us_states_abbr=us_states_abbr[1:]
us_states_abbr.columns=header

In [0]:
us_states_abbr.head()

,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR
5,California,CA


In [0]:
engima_jhu.columns

Out[30]: Index(['fips', 'admin2', 'province_state', 'country_region', 'last_update',
       'latitude', 'longitude', 'confirmed', 'deaths', 'recovered', 'active',
       'combined_key'],
      dtype='object')

In [0]:
us_states_daily.columns

Out[31]: Index(['date', 'state', 'positive', 'probablecases', 'negative', 'pending',
       'totaltestresultssource', 'totaltestresults', 'hospitalizedcurrently',
       'hospitalizedcumulative', 'inicucurrently', 'inicucumulative',
       'onventilatorcurrently', 'onventilatorcumulative', 'recovered',
       'lastupdateet', 'datemodified', 'checktimeet', 'death', 'hospitalized',
       'hospitalizeddischarged', 'datechecked', 'totaltestsviral',
       'positivetestsviral', 'negativetestsviral', 'positivecasesviral',
       'deathconfirmed', 'deathprobable', 'totaltestencountersviral',
       'totaltestspeopleviral', 'totaltestsantibody', 'positivetestsantibody',
       'negativetestsantibody', 'totaltestspeopleantibody',
       'positivetestspeopleantibody', 'negativetestspeopleantibody',
       'totaltestspeopleantigen', 'positivetestspeopleantigen',
       'totaltestsantigen', 'positivetestsantigen', 'fips', 'positiveincrease',
       'negativeincrease', 'total', 'totaltestresultsin

In [0]:
factcovid1=engima_jhu[['fips','province_state','country_region','confirmed','deaths','recovered','active']]
factcovid2=us_states_daily[['fips','date','positive','negative','hospitalized','hospitalizeddischarged']]
factcovid=pd.merge(factcovid1,factcovid2,on='fips',how='inner')

In [0]:
dimRegion1=engima_jhu[['fips','province_state','country_region','latitude', 'longitude']]
dimRegion2=us_county[['fips','county','state']]
dimRegion=pd.merge(dimRegion1,dimRegion2,on='fips',how='inner')

In [0]:
dimDate=us_states_daily[['fips','date']]

In [0]:
dimDate['date']=pd.to_datetime(dimDate['date'],format='%Y%m%d')

<command-1384182662258695>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date']=pd.to_datetime(dimDate['date'],format='%Y%m%d')


In [0]:
dimDate['Year']=dimDate['date'].dt.year
dimDate['Month']=dimDate['date'].dt.month
dimDate['Day']=dimDate['date'].dt.dayofweek

<command-1384182662258696>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['Year']=dimDate['date'].dt.year
<command-1384182662258696>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['Month']=dimDate['date'].dt.month
<command-1384182662258696>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [0]:
us_hospital_beds.columns

Out[44]: Index(['objectid', 'hospital_name', 'hospital_type', 'hq_address',
       'hq_address1', 'hq_city', 'hq_state', 'hq_zip_code', 'county_name',
       'state_name', 'state_fips', 'cnty_fips', 'fips', 'num_licensed_beds',
       'num_staffed_beds', 'num_icu_beds', 'adult_icu_beds', 'pedi_icu_beds',
       'bed_utilization', 'avg_ventilator_usage',
       'potential_increase_in_bed_capac', 'latitude', 'longtitude'],
      dtype='object')

In [0]:
dimHospital=us_hospital_beds[['fips','state_name','latitude','longtitude','hq_address','hospital_type','hospital_name','hq_city', 'hq_state',]]

In [0]:
bucket='covid-19-data-ruthvik'
csv_buffer=StringIO()
factcovid.to_csv(csv_buffer)
s3_resource=boto3.resource('s3',
        aws_access_key_id= AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
    )
s3_resource.Object(bucket,'output/factcovid.csv').put(Body=csv_buffer.getvalue())

Out[34]: {'ResponseMetadata': {'RequestId': '2EM7E6JH10TDNQ3E',
  'HostId': 'x3vRNiCje+/q1/BvtJhymKPqkleCs0wSOzB5iSjKURLDtmh649T4WS4VmxJLW/1mhLbgccIko5g=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'x3vRNiCje+/q1/BvtJhymKPqkleCs0wSOzB5iSjKURLDtmh649T4WS4VmxJLW/1mhLbgccIko5g=',
   'x-amz-request-id': '2EM7E6JH10TDNQ3E',
   'date': 'Sat, 04 Jun 2022 13:46:12 GMT',
   'x-amz-version-id': 'vYLtGVCwO2p.wYGobTgTLcM_t.5jRB_N',
   'etag': '"0957fce49ee019db435496b01bb8a97c"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"0957fce49ee019db435496b01bb8a97c"',
 'VersionId': 'vYLtGVCwO2p.wYGobTgTLcM_t.5jRB_N'}

In [0]:
bucket='covid-19-data-ruthvik'
csv_buffer=StringIO()
dimRegion.to_csv(csv_buffer)
s3_resource=boto3.resource('s3',
        aws_access_key_id= AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
    )
s3_resource.Object(bucket,'output/dimRegion.csv').put(Body=csv_buffer.getvalue())

Out[39]: {'ResponseMetadata': {'RequestId': '9Q15K28VPCP541Q6',
  'HostId': 'fBrmJOwlCwXtdhw5jJbyKezdsSfk1o57HSMO2RLotvJrF2AzczIBxWjWQDjCCPuF2s+Lfy2nuN4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'fBrmJOwlCwXtdhw5jJbyKezdsSfk1o57HSMO2RLotvJrF2AzczIBxWjWQDjCCPuF2s+Lfy2nuN4=',
   'x-amz-request-id': '9Q15K28VPCP541Q6',
   'date': 'Sat, 04 Jun 2022 13:52:40 GMT',
   'x-amz-version-id': 'p0fVAJvcPvyc4taxTkfIAI4A5OOUvryf',
   'etag': '"b862b97dd5eb562a623fb435614577da"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"b862b97dd5eb562a623fb435614577da"',
 'VersionId': 'p0fVAJvcPvyc4taxTkfIAI4A5OOUvryf'}

In [0]:
bucket='covid-19-data-ruthvik'
csv_buffer=StringIO()
dimDate.to_csv(csv_buffer)
s3_resource=boto3.resource('s3',
        aws_access_key_id= AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
    )
s3_resource.Object(bucket,'output/dimDate.csv').put(Body=csv_buffer.getvalue())

Out[36]: {'ResponseMetadata': {'RequestId': '7AABHFHDRCQHHQ9K',
  'HostId': 'wPRaVnZ+Qya2ObQ+PwgscQR3DJIwYxUFWpwRWMamdCMY9E5WVV4K93zLsIHrauKSkIMBRV15RmHRhBXioDA0vA==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'wPRaVnZ+Qya2ObQ+PwgscQR3DJIwYxUFWpwRWMamdCMY9E5WVV4K93zLsIHrauKSkIMBRV15RmHRhBXioDA0vA==',
   'x-amz-request-id': '7AABHFHDRCQHHQ9K',
   'date': 'Sat, 04 Jun 2022 13:47:49 GMT',
   'x-amz-version-id': 'jroI.bQhHm6.rsqh_LCf4zGVvYorByXJ',
   'etag': '"dbda3d14968bb070f9362366103ee958"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"dbda3d14968bb070f9362366103ee958"',
 'VersionId': 'jroI.bQhHm6.rsqh_LCf4zGVvYorByXJ'}

In [0]:
bucket='covid-19-data-ruthvik'
csv_buffer=StringIO()
dimHospital.to_csv(csv_buffer)
s3_resource=boto3.resource('s3',
        aws_access_key_id= AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
    )
s3_resource.Object(bucket,'output/dimHospital.csv').put(Body=csv_buffer.getvalue())

Out[38]: {'ResponseMetadata': {'RequestId': '9HYXN15WXVSJGNPP',
  'HostId': 'QdOEEoW21gqvfnmwvHkbMBDS8925REQrTIrUaY48fqoPW1OK8hjjvu/GyZ3ksjFCrQ5Pa9PqmKo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'QdOEEoW21gqvfnmwvHkbMBDS8925REQrTIrUaY48fqoPW1OK8hjjvu/GyZ3ksjFCrQ5Pa9PqmKo=',
   'x-amz-request-id': '9HYXN15WXVSJGNPP',
   'date': 'Sat, 04 Jun 2022 13:47:59 GMT',
   'x-amz-version-id': '0yZpoBUR7oABN1OpDLaqHfwa4NPj_rNm',
   'etag': '"8e0edcb52b3d691ff0864e4b2e6d35aa"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"8e0edcb52b3d691ff0864e4b2e6d35aa"',
 'VersionId': '0yZpoBUR7oABN1OpDLaqHfwa4NPj_rNm'}

In [0]:
dimDate

,fips,date,Year,Month,Day
0,2,2021-03-07,2021,3,6
1,1,2021-03-07,2021,3,6
2,5,2021-03-07,2021,3,6
3,60,2021-03-07,2021,3,6
4,4,2021-03-07,2021,3,6
...,...,...,...,...,...
20775,53,2020-01-17,2020,1,4
20776,53,2020-01-16,2020,1,3
20777,53,2020-01-15,2020,1,2
20778,53,2020-01-14,2020,1,1


In [0]:
dimHospitalsql=pd.io.sql.get_schema(dimHospital.reset_index(),'dimHospital')
print(dimHospitalsql)

dimRegionsql=pd.io.sql.get_schema(dimRegion.reset_index(),'dimRegion')
print(dimRegionsql)

dimDatesql=pd.io.sql.get_schema(dimDate.reset_index(),'dimDate')
print(dimDatesql)

factcovidsql=pd.io.sql.get_schema(factcovid.reset_index(),'factcovid')
print(factcovidsql)

CREATE TABLE "dimHospital" (
"index" INTEGER,
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hq_address" TEXT,
  "hospital_type" TEXT,
  "hospital_name" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)
CREATE TABLE "dimRegion" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)
CREATE TABLE "dimDate" (
"index" INTEGER,
  "fips" INTEGER,
  "date" TIMESTAMP,
  "Year" INTEGER,
  "Month" INTEGER,
  "Day" INTEGER
)
CREATE TABLE "factcovid" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" REAL,
  "negative" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)


In [0]:
import redshift_connector

In [0]:
conn= redshift_connector.connect(
    host="redshift-cluster-1.cab4qdhkabx2.us-east-2.redshift.amazonaws.com",
    database="dev",
    user="awsuser",
    password="Password123"
)

In [0]:
conn.autocommit=True

In [0]:
cursor=redshift_connector.Cursor=conn.cursor()

In [0]:
cursor.execute(""" CREATE TABLE "dimHospital" (
"index" INTEGER,
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hq_address" TEXT,
  "hospital_type" TEXT,
  "hospital_name" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)
""")

cursor.execute("""
 CREATE TABLE "dimRegion" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)

""")

cursor.execute("""
 CREATE TABLE "dimDate" (
"index" INTEGER,
  "fips" INTEGER,
  "date" TIMESTAMP,
  "Year" INTEGER,
  "Month" INTEGER,
  "Day" INTEGER
)
""")

cursor.execute("""
 CREATE TABLE "factcovid" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" REAL,
  "negative" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)
""")

Out[89]: <redshift_connector.cursor.Cursor at 0x7fae26e22700>

In [0]:
cursor.execute("""
    copy dimDate from 's3://covid-19-data-ruthvik/output/dimDate.csv'
    credentials 'aws_iam_role=arn:aws:iam::528027818862:role/service-role/AmazonRedshift-CommandsAccessRole-20220528T003050'
    delimiter ','
    region 'us-east-2'
    IGNOREHEADER 1
""")

Out[90]: <redshift_connector.cursor.Cursor at 0x7fae26e22700>

In [0]:
cursor.execute("""
    copy dimHospital from 's3://covid-19-data-ruthvik/output/dimHospital.csv'
    credentials 'aws_iam_role=arn:aws:iam::528027818862:role/service-role/AmazonRedshift-CommandsAccessRole-20220528T003050'
    delimiter ','
    region 'us-east-2'
    IGNOREHEADER 1
""")

Out[91]: <redshift_connector.cursor.Cursor at 0x7fae26e22700>

In [0]:
cursor.execute("""
    copy dimRegion from 's3://covid-19-data-ruthvik/output/dimRegion.csv'
    credentials 'aws_iam_role=arn:aws:iam::528027818862:role/service-role/AmazonRedshift-CommandsAccessRole-20220528T003050'
    delimiter ','
    region 'us-east-2'
    IGNOREHEADER 1
""")

Out[92]: <redshift_connector.cursor.Cursor at 0x7fae26e22700>

In [0]:
cursor.execute("""
    copy factcovid from 's3://covid-19-data-ruthvik/output/factcovid.csv'
    credentials 'aws_iam_role=arn:aws:iam::528027818862:role/service-role/AmazonRedshift-CommandsAccessRole-20220528T003050'
    delimiter ','
    region 'us-east-2'
    IGNOREHEADER 1
""")

Out[93]: <redshift_connector.cursor.Cursor at 0x7fae26e22700>